# Introduction

This notebook details the pre-processing of the dataset before analysis / modelling is performed. 

In this notebook we will : 

1. Ingest the raw data from the source
2. Remove any unnecessary fields
3. Save the cleaed up table to an `sqlite` database (within the same repo for reproducibility)

In [1]:
# Library Imports
import sqlite3 
import pandas as pd
import numpy as np
import seaborn as sns

# Data Ingestion

Data is taken from [kaggle](https://www.kaggle.com/datasets/ukveteran/icu-patients/code). The end goal of this project is to predict whether or not a patient survives. 

The raw CSV file (and all other data-related files) is saved in `ICU_patients/datasets/`

In [2]:
# Read in the raw CSV file
df = pd.read_csv("../datasets/ICU_raw.csv")
# View some results
df.head(5)

,Unnamed: 0,ID,Survive,Age,AgeGroup,Sex,Infection,SysBP,Pulse,Emergency
0,1,4,0,87,3,1,1,80,96,1
1,2,8,1,27,1,1,1,142,88,1
2,3,12,1,59,2,0,0,112,80,1
3,4,14,1,77,3,0,0,100,70,0
4,5,27,0,76,3,1,1,128,90,1


Immediately we can see that the columns `Unnamed: 0` & `ID` are purely administrative and provide no useful information. Hence these two columns should be removed.

`AgeGroup` appears to be a binned version of the `Age` column. Feeding both these columns to a model would be unwise since they contain essentially the same information. Keep them for now and decide which to use later on. 

In [3]:
# Drop the admin columns
df.drop(['Unnamed: 0', 'ID'], axis = 1, inplace=True)

Before saving the cleaned up dataframe, it is worth taking a look at the fields to see if there are any missing values. 

In such a case, there are several techniques we could employ (eg. Interpolation / Row Deletion / Forward Fill)

In [4]:
# Obtain the proportion of missing values for each column
df.isnull().sum() * 100 / len(df)

Survive      0.0
Age          0.0
AgeGroup     0.0
Sex          0.0
Infection    0.0
SysBP        0.0
Pulse        0.0
Emergency    0.0
dtype: float64

Fortunately the dataset is clean with no missing values. Now we can move on to saving it

## Saving to database

In [6]:
# Create Sqlite database
database = "../datasets/ICU_data.sqlite"
# make connection. This will also create the sqlite file in the set location
conn = sqlite3.connect(database)

In [7]:
# Save dataframe to database as a table. Save without Index. 
df.to_sql(name='ICU_data', con=conn, index=False)
# Connection Success!

200

In [8]:
# Close the connection
conn.close()

## Reading fom database

We can read from the `sqlite` database using native SQL queries in pandas

[Reference](https://datacarpentry.org/python-ecology-lesson/09-working-with-sql/index.html)

In [9]:
df.head()

,Survive,Age,AgeGroup,Sex,Infection,SysBP,Pulse,Emergency
0,0,87,3,1,1,80,96,1
1,1,27,1,1,1,142,88,1
2,1,59,2,0,0,112,80,1
3,1,77,3,0,0,100,70,0
4,0,76,3,1,1,128,90,1


In [12]:
# Select 10 rows of two columns
conn = sqlite3.connect("../datasets/ICU_data.sqlite")
cur = conn.cursor()

In [13]:
# Return all results of query
cur.execute(\
"""
SELECT 
    Survive
    , Age
FROM ICU_data
LIMIT 10
""")
cur.fetchall()

[(0, 87),
 (1, 27),
 (1, 59),
 (1, 77),
 (0, 76),
 (1, 54),
 (1, 87),
 (1, 69),
 (1, 63),
 (1, 30)]

In [14]:
# Or save the results as a dataframe
tdf = pd.read_sql_query(\
"""
SELECT 
    Survive
    , Age
FROM ICU_data
LIMIT 10
"""
, con = conn)
conn.close()
tdf

,Survive,Age
0,0,87
1,1,27
2,1,59
3,1,77
4,0,76
5,1,54
6,1,87
7,1,69
8,1,63
9,1,30
